## Instagram
We can extract all comments from a specific post.

In [41]:
import os
import time
import random
import openpyxl
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Load Instagram credentials from environment variables (safer approach)
USERNAME = os.getenv("INSTA_USERNAME", "sad*****") # username
PASSWORD = os.getenv("INSTA_PASSWORD", "abc*****") # password 

# Instagram Post URL (Replace with the actual post)
POST_URL = "https://www.instagram.com/reel/DHqmr0dBamQ/?utm_source=ig_web_copy_link"

# WebDriver options to minimize bot detection
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def login():
    """Logs into Instagram"""
    driver.get("https://www.instagram.com/accounts/login/")
    time.sleep(random.uniform(5, 7))  # Randomized delay

    try:
        username_input = driver.find_element(By.NAME, "username")
        password_input = driver.find_element(By.NAME, "password")

        username_input.send_keys(USERNAME)
        password_input.send_keys(PASSWORD)
        password_input.send_keys(Keys.RETURN)

        time.sleep(random.uniform(5, 8))  # Wait for login to process

    except Exception as e:
        print(f"Login failed: {e}")
        driver.quit()

def scrape_post():
    """Scrapes likes and comments from a post"""
    driver.get(POST_URL)
    time.sleep(random.uniform(4, 6))

    # Extract likes
    likes = "N/A"
    try:
        likes_element = driver.find_element(By.XPATH, "//section//span[contains(text(),' likes')]")
        likes = likes_element.text.split()[0]
    except Exception:
        print("Likes count not found.")

    # Click on comments section if available
    try:
        comment_section = driver.find_element(By.XPATH, "//span[contains(text(),' comments')]")
        comment_section.click()
        time.sleep(random.uniform(3, 5))
    except:
        print("No comments found.")

    # Extract comments
    comments_data = []
    try:
        comments = driver.find_elements(By.XPATH, "//ul[@role='presentation']//li")
        for comment in comments:
            try:
                username = comment.find_element(By.XPATH, ".//span/a").text
                comment_text = comment.find_element(By.XPATH, ".//span[@dir='auto']").text
                comments_data.append((username, comment_text))
            except:
                continue
    except:
        print("Error extracting comments.")

    return likes, comments_data

def save_to_excel(likes, comments_data):
    """Saves extracted data to an Excel file"""
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Instagram Data"

    # Headers
    sheet.append(["Likes", "Username", "Comment"])

    # Add likes count
    sheet.append([likes, "", ""])

    # Add comments
    for username, comment in comments_data:
        sheet.append(["", username, comment])

    # Save file
    workbook.save("Instagram_Post_Data2.xlsx")

# Run script
login()
likes, comments_data = scrape_post()
save_to_excel(likes, comments_data)

# Close browser
driver.quit()
print("Data extracted and saved successfully!")


No comments found.
Data extracted and saved successfully!


In [42]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("Instagram_Post_Data2.xlsx")

# Display the first few rows
print(df.head())


   Likes  Username  Comment
0     29       NaN      NaN
